# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [100]:
import sqlite3

In [101]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [102]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
bons_buveurs2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: QTE(NUM)
buveurs_achats2:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: QTE(NUM)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
michel:
  0: michelId(INTEGER) *1
place:
  0: LIEU(TEXT)
  1: REGION(TEXT)
production:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
quantity:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
sales:
  0: NB(NUM)
  1: DATES(NUM)
  2: LIEU(TEX

Again, we will use **`%%sql`** magic for our queries

In [103]:
%load_ext sql
%sql sqlite:///wine.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [104]:
%%sql
select *
FROM MASTER1
limit 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


In [105]:
%%sql
DROP TABLE IF EXISTS wine;
CREATE TABLE wine AS
SELECT DISTINCT NV, CRU, DEGRE, MILL 
FROM Master1
WHERE NV IS NOT NULL;

%%sql 
DROP TABLE IF EXISTS production;
CREATE TABLE production AS
SELECT DISTINCT NP, NOM, PRENOM, REGION 
FROM Master1
WHERE NP IS NOT NULL;

%%sql 
DROP TABLE IF EXISTS quantity;
CREATE TABLE quantity AS
SELECT DISTINCT NP, NV, QTE 
FROM Master1
WHERE NP IS NOT NULL AND NV IS NOT NULL;


 * sqlite:///wine.db
Done.
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql 
DROP TABLE IF EXISTS production;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [106]:
%%sql
select *
FROM quantity, wine, production
limit 5;

 * sqlite:///wine.db
Done.


NP,NV,QTE,NV_1,CRU,DEGRE,MILL,NP_1,NOM,PRENOM,REGION
1,1,300,1,Mercurey,11.5,1980,3,Six,Paul,Alsace
1,1,300,1,Mercurey,11.5,1980,6,Marmagne,Bernard,Bourgogne
1,1,300,1,Mercurey,11.5,1980,8,Lioger d'Harduy,Gabriel,Bourgogne
1,1,300,1,Mercurey,11.5,1980,16,Barbin,Bernard,Bourgogne
1,1,300,1,Mercurey,11.5,1980,17,Faiveley,Guy,Bourgogne


In [107]:
%%sql
DROP TABLE IF EXISTS customer;
CREATE TABLE customer AS
SELECT DISTINCT NB, NOM, PRENOM, TYPE 
FROM Master2
WHERE NB IS NOT NULL;

%%sql 
DROP TABLE IF EXISTS place;
CREATE TABLE place AS
SELECT DISTINCT LIEU, REGION 
FROM Master2
WHERE LIEU IS NOT NULL AND REGION IS NOT NULL;

%%sql 
DROP TABLE IF EXISTS sales;
CREATE TABLE sales AS
SELECT DISTINCT NB, DATES, LIEU, QTE, NV 
FROM Master2
WHERE LIEU IS NOT NULL AND NB IS NOT NULL;

%%sql DROP TABLE IF EXISTS dummy;
-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.
(sqlite3.OperationalError) near "%": syntax error
[SQL: %%sql 
DROP TABLE IF EXISTS place;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [108]:
%%sql
select *
FROM place
limit 5;

 * sqlite:///wine.db
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [109]:
%%sql
DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT nb, nom, prenom, type  
FROM customer
WHERE type="gros" OR type="moyen";

 * sqlite:///wine.db
Done.
Done.


[]

In [110]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb LIMIT 10;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


### Correction 1.1

In [111]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [67]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT customer.nb, customer.nom, customer.prenom, customer.type  
FROM customer LEFT JOIN sales ON sales.nb = customer.nb 
WHERE sales.QTE is null ;

 * sqlite:///wine.db
Done.
Done.


[]

In [68]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb LIMIT 10;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


### Correction 1.2

In [69]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [113]:
%%sql
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT customer.nb, customer.nom, customer.prenom, customer.type, QTE  
FROM customer JOIN sales ON sales.nb = customer.nb 
WHERE sales.QTE != 0
GROUP BY customer.nb;


 * sqlite:///wine.db
Done.
Done.


[]

In [114]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb limit 10;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,QTE
1,Aristote,None,petit,78
2,Artaud,Antonin,moyen,33
3,Aron,Raymond,gros,36
4,Apollinaire,Guillaume,moyen,24
5,Audiberti,Jacques,petit,20
6,Arrabal,Fernando,gros,36
7,Anouilh,Jean,moyen,6
8,Aragon,Louis,gros,12
9,Ajar,Emile,petit,10
10,Andersen,Yann,gros,1


### Correction 1.3

In [115]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,QTE
1,Aristote,None,petit,78
2,Artaud,Antonin,moyen,33
3,Aron,Raymond,gros,36
4,Apollinaire,Guillaume,moyen,24
5,Audiberti,Jacques,petit,20
6,Arrabal,Fernando,gros,36
7,Anouilh,Jean,moyen,6
8,Aragon,Louis,gros,12
9,Ajar,Emile,petit,10
10,Andersen,Yann,gros,1


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [116]:
%%sql
DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT place.LIEU, place.REGION, sales.QTE  
FROM place JOIN sales ON sales.LIEU = place.LIEU 
WHERE sales.DATES = 1983;


 * sqlite:///wine.db
Done.
Done.


[]

In [117]:
# Test
%sql SELECT * FROM q83pl;

 * sqlite:///wine.db
Done.


LIEU,REGION,QTE


In [118]:
%sql SELECT * FROM place;

 * sqlite:///wine.db
Done.


LIEU,REGION
BORDEAUX,NOUVELLE-AQUITAINE
PARIS,ÎLE-DE-FRANCE
RENNES,BRETAGNE
LYON,AUVERGNE-RHÔNE-ALPES
NICE,PROVENCE-ALPES-CÔTE D'AZUR
MERCUREY,BOURGOGNE-FRANCHE-COMTÉ
TOULOUSE,OCCITANIE
SENS,BOURGOGNE-FRANCHE-COMTÉ
LILLE,HAUTS-DE-FRANCE
NANCY,GRAND EST


### Correction 1.4

In [13]:
# Test
%sql SELECT * FROM q83pl;

Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [119]:
%%sql
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs;

 * sqlite:///wine.db
Done.
Done.


[]

In [120]:
%%sql
select * 
from RBB
limit 10

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


### Correction 2.1

#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [121]:
%%sql
DROP TABLE IF EXISTS bons_buveurs;
CREATE TABLE bons_buveurs AS
SELECT customer.nb, customer.nom, customer.prenom, customer.type, QTE  
FROM customer JOIN sales ON sales.nb = customer.nb 
WHERE sales.QTE != 0
GROUP BY customer.nb;


 * sqlite:///wine.db
(sqlite3.OperationalError) use DROP VIEW to delete view bons_buveurs
[SQL: DROP TABLE IF EXISTS bons_buveurs;]
(Background on this error at: http://sqlalche.me/e/e3q8)


In [122]:
%%sql
UPDATE customer
SET TYPE = 'HUGE'
WHERE
NB IN 
(SELECT sales.NB AS total FROM customer 
JOIN sales ON sales.NB = customer.NB
GROUP BY sales.NB
HAVING SUM(QTE) >= 100 AND customer.TYPE NOT LIKE 'HUGE')

 * sqlite:///wine.db
6 rows affected.


[]

In [123]:
%%sql 
select *
FROM customer

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


Update instances

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [124]:
%%sql
SELECT RBB.*, bons_buveurs.*
FROM RBB
    JOIN bons_buveurs ON (RBB.NB = bons_buveurs.NB)
WHERE RBB.NB IS NULL OR bons_buveurs.NB IS NULL

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,NB_1,NOM_1,PRENOM_1,TYPE_1


In [125]:
%%sql
select * from RBB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


In [126]:
%%sql 
select * from bons_buveurs

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [127]:
%%sql
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;

 * sqlite:///wine.db
Done.
Done.


[]

In [128]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [129]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM,TYPE) 
VALUES ("101", "DESSE", "MATTHIEU","GROS")

 * sqlite:///wine.db
1 rows affected.


[]

In [130]:
# Test
%sql SELECT * FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [131]:
%%sql
select * 
from buveurs_asec

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


In [88]:
%%sql
SELECT RBA.*, buveurs_asec.*
FROM RBA
    JOIN buveurs_asec ON (RBA.NB = buveurs_asec.NB)
WHERE RBA.NB IS NULL OR buveurs_asec.NB IS NULL

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,NB_1,NOM_1,PRENOM_1,TYPE_1


___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

Answer: 
First, we have to create your tables to make link between people (User, relationship and friend).
Then, we have to build the section post with 3 others tables : post (cointaing all informations of a post), comment and other (containing information about the user post).

![](DB_design.jpg)

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.